In [4]:
def plot_iteration_result(theta_a, it, res, sols):
    print(r"The slip angle of %.2f ° was found after %i iterations. The corresponding earth pressure is %.2f kN/m." 
      %(np.rad2deg(theta_a),it,Ea(theta_a)/1.e3))
    fig, ax = plt.subplots(ncols=2,figsize=(18,8))
    thetas = np.linspace(np.pi/6,np.pi/2,100)
    ax[0].plot(np.rad2deg(thetas),Ea(thetas)/1e3,label=r'$E_\mathrm{a}(\vartheta)$')
    ax[0].plot(np.rad2deg(sols),Ea(sols)/1e3,marker='+',ls='',markersize=6,color='green',label=r'$E_\mathrm{a}^i$')
    ax[0].plot(np.rad2deg(theta_a),Ea(theta_a)/1e3,marker='d',color='red',label=r'$E_\mathrm{a}$')
    ax[0].set_xlabel(r'$\vartheta$ / °')
    ax[0].legend()
    ax[0].set_ylabel(r'$E_\mathrm{a}$ / kN m$^{-1}$')
    
    ax[1].plot(np.abs(res/res[0]))
    ax[1].plot([len(res)-1+0.1,len(res)-2+0.1],[np.abs(res[-1]/res[0]),np.sqrt(np.abs(res[-1]/res[0]))],ls='--',color='black')
    ax[1].text(len(res)-1+0.1,(np.abs(res[-1]/res[0])+np.sqrt(np.abs(res[-1]/res[0])))/2,'-2:1')
    ax[1].set_xlabel(r'$n_\mathrm{it}$')
    ax[1].set_ylabel(r'$\frac{|| r ||}{|| r_0 ||}$')
    ax[1].set_yscale('log')

In [4]:
def plot_iterative_interactive():
    @interact(theta0=widgets.BoundedFloatText(value=45,min=10,max=90,description=r'$\vartheta_0 / °$'))
    def plotme(theta0):
        theta_a, it, res, sols = find_theta(np.deg2rad(theta0))
        print("The slip angle of %.2f ° was found after %i iterations. The corresponding earth pressure is %.2f kN/m." 
          %(np.rad2deg(theta_a),it,Ea(theta_a)/1.e3))
        fig, ax = plt.subplots(ncols=2,figsize=(18,8))
        thetas = np.linspace(np.pi/6,np.pi/2,100)
        ax[0].plot(np.rad2deg(thetas),Ea(thetas)/1e3,label=r'$E_\mathrm{a}(\vartheta)$')
        ax[0].plot(np.rad2deg(sols),Ea(sols)/1e3,marker='+',ls='',markersize=6,color='green',label=r'$E_\mathrm{a}^i$')
        ax[0].plot(np.rad2deg(theta_a),Ea(theta_a)/1e3,marker='d',color='red',label=r'$E_\mathrm{a}$')
        ax[0].set_xlabel(r'$\vartheta$ / °')
        ax[0].legend()
        ax[0].set_ylabel(r'$E_\mathrm{a}$ / kN m$^{-1}$')
        #res = np.array([1e-1,1e-2,1e-4,1e-8])
        ax[1].plot(np.abs(res),marker='s',label='observed')
        for i in range(1,len(res)-1):
            order = np.log10(res[i+1]/res[i]) / np.log10(res[i]/res[i-1])
            ax[1].text((i+1+i)/2,10**((np.log10(res[i+1])+np.log10(res[i]))/2)," c.o. " + str(np.round(order,2)))
        #ax[1].plot([len(res)-1+0.1,len(res)-2+0.1],[np.abs(res[-1]/res[0]),np.sqrt(np.abs(res[-1]/res[0]))],ls='--',color='black')
        #ax[1].text(len(res)-1+0.1,(np.abs(res[-1]/res[0])+np.sqrt(np.abs(res[-1]/res[0])))/2,'-2:1')
        ax[1].set_xlabel(r'$n_\mathrm{it}$')
        ax[1].set_ylabel(r'$|| r ||$')
        ax[1].set_yscale('log')

In [ ]:
def plot_interactive_earth_pressure():
    @interact(ic=widgets.IntSlider(min=0, max=50, value=10, step=5, description=r'$c$ / kN m$^{-2}$:'),
              iphi = widgets.IntSlider(min=20, max=45, value=30, step=5, description=r'$\varphi$ / °:'),
              igamma=widgets.IntSlider(min=14, max=25, value=18, step=1, description=r'$\gamma$ / kN m$^{-3}$:'),
              iH=widgets.IntSlider(min=2, max=10, value=5, step=1, description=r'$H$ / m:'),
              ialpha = widgets.IntSlider(min=0, max=15, value=5, step=1, description=r'$\alpha$ / °:'),
              ibeta = widgets.IntSlider(min=0, max=15, value=5, step=1, description=r'$\beta$ / °:'),
              idelta = widgets.IntSlider(min=0, max=20, value=10, step=2, description=r'$\delta_\mathrm{a}$ / °:'),
             )
    def plot_earth_pressure(ic,iphi,igamma,iH,ialpha,ibeta,idelta):
        #unit conversion
        global c
        global gamma
        global phi
        global alpha
        global beta
        global delta
        global H

        H = iH
        c = ic * 1000.
        gamma = igamma * 1000.
        phi = iphi * np.pi/180.
        alpha = ialpha * np.pi/180.
        beta = ibeta * np.pi/180.
        delta = idelta * np.pi/180.
        #Calculation
        theta_a, it, res, sols = find_theta(np.deg2rad(theta_ag_norm(phi,alpha,delta,beta)))

        thetas = np.linspace(np.pi/6,np.pi/2*0.9,100)
        plt.plot(thetas*180/np.pi,Ea(thetas)/1e3)
        plt.plot(theta_a*180/np.pi,Ea(theta_a)/1e3,marker='d',color='red')
        plt.xlabel(r'$\vartheta$ / °')
        plt.ylabel(r'$E_\mathrm{a}$ / kN m$^{-1}$')
        #plt.gca().set_ylim(bottom=0)
        plt.text(theta_a*180/np.pi,Ea(theta_a)/1e3,r'$E_\mathrm{a} = %.2f$ kN m$^{-1}$, $\vartheta$ = %.1f °' %(Ea(theta_a)/1e3,theta_a*180/np.pi))
        plt.show();